In [1]:
from dotenv import load_dotenv
from loguru import logger
import sys

sys.path.append("..")

from news_extraction_pipeline.config import AINewsConfig
from news_extraction_pipeline.pipeline_steps import (
    extract_from_multiple_feed_urls,
    filter_by_date_threshold,
    filter_by_keywords,
    convert_datetime_columns_to_str
)
from utils.io_utils import store_df_to_excel

load_dotenv()

True

In [2]:
news_config = AINewsConfig()

In [4]:
bad_url = "https://techcrunch.com/category/artificial-intelligence/"

In [5]:
logger.info("Starting AI news retrieval process...")

news_config_dict = news_config.model_dump()

news_sources = [val for key, val in news_config_dict.items() if key.endswith("_FEED_URL")]

news_sources = [bad_url, news_config.MIT_NEWS_FEED_URL, news_config.AI_NEWS_FEED_URL]

articles = extract_from_multiple_feed_urls(news_sources)

2025-10-08 23:46:05.933 | INFO     | __main__:<module>:1 - Starting AI news retrieval process...
2025-10-08 23:46:05.935 | INFO     | news_extraction_pipeline.extractors.news.news_extractors:current_feed_url:82 - Setting current feed url to https://techcrunch.com/category/artificial-intelligence/
2025-10-08 23:46:05.938 | INFO     | news_extraction_pipeline.extractors.news.news_extractors:current_feed_url:82 - Setting current feed url to https://news.mit.edu/rss/feed
2025-10-08 23:46:05.938 | WARNING  | news_extraction_pipeline.selectors.extractor_selector:get_extractor:68 - No image extractor found for the url: https://techcrunch.com/category/artificial-intelligence/. The data retrieve will not contain an image_url
2025-10-08 23:46:05.940 | INFO     | news_extraction_pipeline.extractors.news.news_extractors:current_feed_url:82 - Setting current feed url to https://www.artificialintelligence-news.com/artificial-intelligence-news/feed/
2025-10-08 23:46:05.941 | INFO     | news_extractio

In [5]:
articles.info()

<class 'pandas.core.frame.DataFrame'>
Index: 62 entries, 0 to 49
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype              
---  ------        --------------  -----              
 0   title         62 non-null     object             
 1   news_link     62 non-null     object             
 2   image_link    57 non-null     object             
 3   publish_date  62 non-null     datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1), object(3)
memory usage: 2.4+ KB


In [6]:
articles.sample(10)

,title,news_link,image_link,publish_date
16,AI maps how a new antibiotic targets gut bacteria,https://news.mit.edu/2025/ai-maps-how-new-anti...,https://news.mit.edu/sites/default/files/style...,2025-10-03 21:00:00+00:00
45,MIT’s work with Idaho National Laboratory adva...,https://news.mit.edu/2025/mit-work-with-idaho-...,https://news.mit.edu/sites/default/files/style...,2025-09-23 13:00:00+00:00
19,Martin Trust Center for MIT Entrepreneurship w...,https://news.mit.edu/2025/martin-trust-center-...,https://news.mit.edu/sites/default/files/style...,2025-10-02 19:55:00+00:00
33,A beacon of light,https://news.mit.edu/2025/beacon-of-light-geol...,https://news.mit.edu/sites/default/files/style...,2025-09-29 20:00:00+00:00
35,How the brain splits up vision without you eve...,https://news.mit.edu/2025/how-brain-splits-vis...,https://news.mit.edu/sites/default/files/style...,2025-09-26 19:50:00+00:00
2,"Riccardo Comin, two MIT alumni named 2025 Moor...",https://news.mit.edu/2025/riccardo-comin-mit-a...,https://news.mit.edu/sites/default/files/style...,2025-10-08 19:00:00+00:00
3,How AI is changing the way we travel,https://www.artificialintelligence-news.com/ne...,https://www.artificialintelligence-news.com/wp...,2025-10-07 11:00:00+00:00
6,China Mobile Shanghai launches industry-first ...,https://www.artificialintelligence-news.com/ne...,https://www.artificialintelligence-news.com/wp...,2025-10-03 09:00:00+00:00
4,Using generative AI to diversify virtual train...,https://news.mit.edu/2025/using-generative-ai-...,https://news.mit.edu/sites/default/files/style...,2025-10-08 17:45:00+00:00
21,A simple formula could guide the design of fas...,https://news.mit.edu/2025/simple-formula-could...,https://news.mit.edu/sites/default/files/style...,2025-10-02 18:00:00+00:00


In [6]:
filtered_by_date = filter_by_date_threshold(
    df = articles,
    filter_column=news_config.DATE_COLUMN,
    max_days_old=10
)


2025-10-08 23:47:18.702 | INFO     | news_extraction_pipeline.pipeline_steps:filter_by_date_threshold:162 - Filtering articles published within the last 10 days.
2025-10-08 23:47:18.705 | INFO     | news_extraction_pipeline.pipeline_steps:filter_by_date_threshold:175 - Date filtering complete. 47 articles published within the allowed range.


In [7]:
filtered_by_kw = filter_by_keywords(
    df = filtered_by_date,
    filter_column=news_config.COLUMN_TO_FILTER_BY_KW,
    case_sen_search_kw=news_config.CASE_SEN_SEARCH_KW,
    case_insen_search_kw=news_config.CASE_INSEN_SEARCH_KW,
)

2025-10-08 23:47:21.154 | DEBUG    | news_extraction_pipeline.pipeline_steps:filter_by_keywords:107 - Filtering articles by the next parameters...
2025-10-08 23:47:21.155 | DEBUG    | news_extraction_pipeline.pipeline_steps:filter_by_keywords:108 - case_sen_search_kw =[' AI ', 'AI ', 'AI ', 'A.I.', ' AI-', 'AI-']
2025-10-08 23:47:21.156 | DEBUG    | news_extraction_pipeline.pipeline_steps:filter_by_keywords:109 - case_insen_search_kw =['Artificial Intelligence', 'Machine Learning', 'Deep Learning', 'Neural Networks', 'NLP', 'Computer Vision', 'Data Science', 'Gemini', 'Bard', 'ChatGPT', 'GPT-4', 'DALL-E', 'MidJourney', 'Stable Diffusion', 'Claude', 'LLaMA', 'Whisper']
2025-10-08 23:47:21.157 | DEBUG    | news_extraction_pipeline.pipeline_steps:filter_by_keywords:110 - Filtering by column: title
2025-10-08 23:47:21.158 | INFO     | news_extraction_pipeline.pipeline_steps:filter_by_keywords:126 - Keyword filtering complete. 14 articles matched the criteria.


In [9]:
final_df = convert_datetime_columns_to_str(
    df = filtered_by_kw,
    string_format = news_config.DATE_STRING_FORMAT
)

final_df.info()

2025-10-08 23:47:33.663 | INFO     | news_extraction_pipeline.pipeline_steps:convert_datetime_columns_to_str:214 - Converting datetime columns to string format: %Y-%m-%dT%H:%M:%SZ
2025-10-08 23:47:33.664 | DEBUG    | news_extraction_pipeline.pipeline_steps:convert_datetime_columns_to_str:215 - Detected datetime columns: ['publish_date']
2025-10-08 23:47:33.665 | INFO     | news_extraction_pipeline.pipeline_steps:convert_datetime_columns_to_str:220 - Datetime conversion complete.


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         14 non-null     object
 1   news_link     14 non-null     object
 2   image_link    14 non-null     object
 3   publish_date  14 non-null     object
dtypes: object(4)
memory usage: 580.0+ bytes


In [10]:
store_df_to_excel(
    df = final_df,
    local_file_path=news_config.FILE_PATH,
    sheet_name=news_config.EXCEL_SHEET_NAME,
    table_name=news_config.EXCEL_TABLE_NAME
)